In [0]:
import numpy as np
import pandas as pd

In [7]:
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv

--2018-09-01 03:16:46--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12564851 (12M) [text/plain]
Saving to: ‘train_NIR5Yl1.csv’

train_NIR5Yl1.csv   100%[===================>]  11.98M  26.1MB/s    in 0.5s    

2018-09-01 03:16:46 (26.1 MB/s) - ‘train_NIR5Yl1.csv’ saved [12564851/12564851]

--2018-09-01 03:16:47--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response

In [0]:
data= pd.read_csv("train_NIR5Yl1.csv")

In [9]:
data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [0]:
data.drop("ID",axis =1,inplace=True)

In [11]:
data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes
0,a,3942.0,2.0,155623,7855.0,42.0
1,a,26046.0,12.0,21781,55801.0,1175.0
2,c,1358.0,4.0,56177,8067.0,60.0
3,a,264.0,3.0,168793,27064.0,9.0
4,c,4271.0,4.0,112223,13986.0,83.0


In [12]:
data_dummies=pd.get_dummies(data["Tag"])
data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes
0,a,3942.0,2.0,155623,7855.0,42.0
1,a,26046.0,12.0,21781,55801.0,1175.0
2,c,1358.0,4.0,56177,8067.0,60.0
3,a,264.0,3.0,168793,27064.0,9.0
4,c,4271.0,4.0,112223,13986.0,83.0


In [13]:
data = pd.concat([data,pd.get_dummies(data['Tag'], drop_first=True)], axis=1); data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
data.drop("Tag",axis=1,inplace=True)

In [15]:
data.head()

,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data.drop('Upvotes', axis=1)
y = data['Upvotes']

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=69)

In [19]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(297040, 13)
(297040,)
(33005, 13)
(33005,)


In [0]:
import xgboost as xgb

xg_reg=xgb.XGBRegressor(objective='reg:linear',n_estimators=10,seed=150)
xg_reg.fit(X_train,y_train)
preds=xg_reg.predict(X_test)

In [0]:
from sklearn.metrics import mean_squared_error
rmse=np.sqrt(mean_squared_error(y_test,preds))

In [22]:
print("RMSE : %f"% (rmse))

RMSE : 2276.641399


In [23]:
#tuning and boosting of xg boost

DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg=xgb.train(params=params,dtrain=DM_train,num_boost_round=10)

preds_2=xg_reg.predict(DM_test)

rmse_2=np.sqrt(mean_squared_error(y_test,preds_2))

print("new RMSE= %f" %rmse_2)


new RMSE= 3296.124775


In [24]:
rmse

2276.6413991351656

In [25]:
rmse_2

3296.12477505686

### Trying to apply grid search cv

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


steps = [('clf', xgb.XGBRegressor())]


clf_param_grid = {
    'clf__max_depth' : [5,10,20],
    'clf__learning_rate' : [0.1,0.01],
    'clf__n_jobs' : [4],
    'clf__n_estimators' : [50, 100,]
    
}


xgbclf = Pipeline(steps)

X_new = X.copy()
# X_new.drop('ID',axis=1,inplace=True)
estimator = GridSearchCV(xgbclf, param_grid=clf_param_grid ,cv =4, verbose=2, scoring="neg_mean_squared_error")
estimator.fit(X_new,y)

print(estimator.best_params_)
print(np.sqrt(-1*estimator.best_score_))

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   8.1s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.4s remaining:    0.0s


[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   8.1s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   8.1s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   8.0s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4, total=  16.0s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4, total=  16.0s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jo

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 29.5min finished


{'clf__learning_rate': 0.1, 'clf__max_depth': 5, 'clf__n_estimators': 100, 'clf__n_jobs': 4}
1395.2965873858348


# Attention !!!
#### i have just tried a bit more tuning....
initially i took num_boost_round=200 so it didn't run on my pc,so i reduced it to 2 then only rmse was 2368.but when i just increased to 20 th result is here!
So, try to run it at 200 rounds on gpu or even more. Also, try to change the nfolds.

In [27]:
review_data_matrix=xgb.DMatrix(data=X,label=y)

untuned_params = {"objective":"reg:linear"}

tuned_cv_rmse = xgb.cv(dtrain = review_data_matrix,params=untuned_params, nfold=7, num_boost_round=200, metrics="rmse",as_pandas=True, seed=123)

print(type(tuned_cv_rmse))

print("Tuned rmse: %f" %((tuned_cv_rmse["test-rmse-mean"]).tail(1)))


<class 'pandas.core.frame.DataFrame'>
Tuned rmse: 1365.867336


## trying more tuning methods

I didn't found this method benificial, so just go through it and if possible try hyperparameter tuning.that's it

In [28]:
tuned_params={"objective":"reg:linear",'colsample_bytree':0.3,'learning_rate':0.1,'max_depth':5}

new_tuned_cv_rmse = xgb.cv(dtrain=review_data_matrix, params=tuned_params, nfold=5, num_boost_round=20, metrics="rmse",as_pandas=True, seed=123)

print("Tuned rmse: %f" %((new_tuned_cv_rmse["test-rmse-mean"]).tail(1)))


Tuned rmse: 2976.052100


# SVR

In [0]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma = 0.1)

svr_rbf.fit(X_train,y_train)
svr_rbf.predict(X_test)
svr_rbf.score(X_test,y_test)